In [27]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

In [28]:
import requests

In [42]:
# Function to fetch weather data
def get_weather(city, api_key):
    # url = f"http://api.openweathermap.org/data/3.0/weather?q={city}&appid={api_key}&units=metric"
    url = f"https://api.openweathermap.org/data/3.0/onecall?q={city}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error:", response.status_code)
        return None

# Function to display weather data
def display_weather(data):
    if data:
        city = data["name"]
        temp = data["main"]["temp"]
        weather = data["weather"][0]["description"]
        humidity = data["main"]["humidity"]
        wind_speed = data["wind"]["speed"]

        print(f"Weather in {city}:")
        print(f"Temperature: {temp}°C")
        print(f"Condition: {weather}")
        print(f"Humidity: {humidity}%")
        print(f"Wind Speed: {wind_speed} m/s")
    else:
        print("No data to display.")

# # Main program
# if __name__ == "__main__":
#     city_name = input("Enter the city name: ")
#     api_key = "your_api_key_here"  # Replace with your OpenWeatherMap API key
#     weather_data = get_weather(city_name, api_key)
#     display_weather(weather_data)


In [43]:
city_name = 'Surabaya'

In [44]:
weather_data = get_weather(city_name, api_key)
display_weather(weather_data)

Error: 401
No data to display.


In [32]:
from datetime import date, datetime, timedelta

In [33]:
today = datetime(2025, 1, 19, 9, 00, 00)
print(today - timedelta(days=1))
print((today - timedelta(days=1)).timestamp())
print(int((today - timedelta(days=1)).timestamp()))


2025-01-18 09:00:00
1737165600.0
1737165600


In [84]:
# Testing geocoding to get the latlong data of a city
def get_city_coordinates(city, api_key):
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return lat, lon
        else:
            print(f"City '{city}' not found.")
            return None, None
    else:
        print("Error fetching coordinates:", response.status_code)
        return None, None

city = "Surabaya"
# city = "Jakarta"
lat, lon = get_city_coordinates(city, api_key)
print(lat)
print(lon)

-7.2459717
112.7378266


In [85]:
import time

In [92]:
tz = "+07:00"
date = "2025-01-18"
base_url = "https://api.openweathermap.org/data/2.5/onecall/timemachine"
# base_url = f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={lat}&lon={lon}&date={date}&tz={tz}&appid={api_key}"
# base_url = f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={lat}&lon={lon}&date={date}&tz={tz}&appid={api_key}"
# base_url = f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat=60.45&lon=-38.67&date=2023-03-30&tz=+03:00&appid={api_key}"
base_url
# response = requests.get(base_url, params=params)
# response = requests.get(base_url)
# response

'https://api.openweathermap.org/data/2.5/onecall/timemachine'

In [87]:
def get_historical_weather(lat, lon, api_key, days=30):
    """Fetch historical weather data for the last 'days' days."""
    base_url = "https://api.openweathermap.org/data/2.5/onecall/timemachine"
    weather_data = []

    for i in range(days):
        # Calculate the timestamp for each day
        timestamp = int((datetime.utcnow() - timedelta(days=i+1)).timestamp())
        params = {
            "lat": lat,
            "lon": lon,
            "dt": timestamp,
            "appid": api_key,
            "units": "metric"  # Optional: Metric units for temperature
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            weather_data.append(response.json())
        else:
            print(f"Error fetching weather data for day {i+1}: {response.status_code}")
            break

        # Avoid hitting rate limits
        time.sleep(1)

    return weather_data

In [88]:
def main():
    # api_key = "YOUR_API_KEY"  # Replace with your API key
    city_name = "Surabaya"  # Change to your desired city
    days = 30  # Number of past days to retrieve

    # Step 1: Get city coordinates
    lat, lon = get_city_coordinates(city_name, api_key)
    if lat is None or lon is None:
        return

    # Step 2: Fetch historical weather data
    weather_data = get_historical_weather(lat, lon, api_key, days)

    # Step 3: Display or process the data
    for day in weather_data:
        date = datetime.fromtimestamp(day["current"]["dt"]).strftime('%Y-%m-%d')
        temp = day["current"]["temp"]
        weather = day["current"]["weather"][0]["description"]
        print(f"Date: {date}, Temp: {temp}°C, Weather: {weather}")

if __name__ == "__main__":
    main()

Error fetching weather data for day 1: 401


In [89]:
def fetch_weather_data(city_name, lat, lon, api_key, days=30):
    """Fetch weather data for the last 'days' days."""
    base_url = "https://api.openweathermap.org/data/2.5/onecall/timemachine"
    weather_data = []

    for i in range(days):
        # Calculate the timestamp for each day
        timestamp = int((datetime.utcnow() - timedelta(days=i+1)).timestamp())
        params = {
            "lat": lat,
            "lon": lon,
            "dt": timestamp,
            "appid": api_key,
            "units": "metric"  # Use metric units for temperature
        }

        # Make the API call
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            day_data = response.json()
            weather_data.append({
                "timestamp": timestamp,
                "date": datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d'),
                "temperature": day_data["current"]["temp"],
                "description": day_data["current"]["weather"][0]["description"]
            })
        else:
            print(f"Error fetching data for {datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')}: {response.status_code}")
        
        # Avoid hitting rate limits
        time.sleep(1)

    return weather_data

def main():
    # Fetch weather data for the last 30 days
    weather_data = fetch_weather_data(city, lat, lon, api_key, days=30)

    # Display the results
    for day in weather_data:
        print(f"Date: {day['date']}, Temp: {day['temperature']}°C, Weather: {day['description']}")

if __name__ == "__main__":
    main()

Error fetching data for 2025-01-18: 401
Error fetching data for 2025-01-17: 401
Error fetching data for 2025-01-16: 401
Error fetching data for 2025-01-15: 401


KeyboardInterrupt: 